In [1]:
import cv2
import mediapipe as mp
import numpy as np
import random
import time
import sqlite3
import json
import os
import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt

# -------------------- CONFIGURACIÓN DB --------------------
def init_db():
    preferred_path = r"C:\\Users\\carlo\\Documents\\TFG\\juegos\\Datos Juegos"
    if not os.path.exists(preferred_path):
        preferred_path = os.path.join(os.getcwd(), "Datos Juegos")
    os.makedirs(preferred_path, exist_ok=True)
    db_path = os.path.join(preferred_path, "game_data.db")
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS patients (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE,
        points INTEGER DEFAULT 0,
        rank TEXT DEFAULT 'bronce',
        prestige INTEGER DEFAULT 1
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS sessions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id INTEGER,
        timestamp TEXT,
        release_times TEXT,
        collected_count INTEGER,
        failed_count INTEGER,
        falling_speed INTEGER,
        total_balls INTEGER,
        angle_series TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(id)
    )
    """)
    conn.commit()
    return conn, cursor, preferred_path

# -------------------- GAMIFICACIÓN --------------------
def get_medalla_path(rango):
    base_dir = r"C:\\Users\\carlo\\Documents\\TFG\\juegos\\Imagenes_Juegos"
    ruta_directa = os.path.join(base_dir, f"{rango}.png")
    if os.path.exists(ruta_directa):
        return ruta_directa
    carpeta = os.path.join(base_dir, rango)
    if os.path.exists(carpeta):
        for archivo in os.listdir(carpeta):
            if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
                return os.path.join(carpeta, archivo)
    return None

def actualizar_rango_y_puntos(cursor, conn, patient_id, release_times):
    cursor.execute("SELECT points, rank, prestige FROM patients WHERE id = ?", (patient_id,))
    row = cursor.fetchone()
    if row:
        puntos_actuales, rango_actual, prestigio_actual = row
    else:
        puntos_actuales, rango_actual, prestigio_actual = 0, "bronce", 1

    cursor.execute("""
        SELECT release_times FROM sessions
        WHERE patient_id = ? AND release_times IS NOT NULL AND release_times != '[]'
    """, (patient_id,))
    datos = cursor.fetchall()
    todas_las_velocidades = []
    for (rt,) in datos:
        tiempos = json.loads(rt)
        velocidades = [1/t for t in tiempos if t > 0]
        todas_las_velocidades.extend(velocidades)

    media_actual = np.mean(todas_las_velocidades) if todas_las_velocidades else 0
    puntos_nuevos = sum(1 for t in release_times if t > 0 and (1/t) > media_actual)
    total = puntos_actuales + puntos_nuevos

    nuevo_rango = rango_actual
    nuevo_prestigio = prestigio_actual

    if total >= 45:
        total = 0
        nuevo_prestigio += 1
        nuevo_rango = "bronce"
    elif total >= 30:
        nuevo_rango = "oro"
    elif total >= 15:
        nuevo_rango = "plata"
    else:
        nuevo_rango = "bronce"

    cursor.execute("""
        UPDATE patients
        SET points = ?, rank = ?, prestige = ?
        WHERE id = ?
    """, (total, nuevo_rango, nuevo_prestigio, patient_id))
    conn.commit()
    return nuevo_rango, nuevo_prestigio, total

# -------------------- MOSTRAR MENSAJE Y MEDALLA --------------------
def mostrar_mensaje_resultado(puntos):
    if puntos >= 5:
        mensaje = "OLE OLE y OLE"
    elif puntos >= 3:
        mensaje = "OLE OLE"
    elif puntos >= 1:
        mensaje = "OLE"
    else:
        mensaje = "Buen intento, sigue practicando!"
    messagebox.showinfo("Resultado", mensaje)

def mostrar_imagen_medalla(rango, ventana=None):
    ruta = get_medalla_path(rango)
    if ruta and os.path.exists(ruta):
        img = cv2.imread(ruta)
        if ventana is not None:
            h_img, w_img = img.shape[:2]
            h_v, w_v = ventana.shape[:2]
            x_offset = w_v - w_img - 10
            y_offset = h_v - h_img - 10
            if x_offset >= 0 and y_offset >= 0:
                ventana[y_offset:y_offset + h_img, x_offset:x_offset + w_img] = img
        else:
            cv2.imshow("SUBES DE RANGO!", img)
            cv2.waitKey(3000)
            cv2.destroyWindow("SUBES DE RANGO!")

def mostrar_rango_inicial(rank, prestige, points):
    mensaje = f"Rango actual: {rank}\nPrestigio: {prestige}\nPuntos actuales: {points}/45"
    messagebox.showinfo("Estado del Jugador", mensaje)



# -------------------- FUNCIONES AUXILIARES --------------------
def calculate_hand_angle(landmarks):
    thumb = landmarks[4]
    pinky = landmarks[20]
    dx = pinky.x - thumb.x
    dy = pinky.y - thumb.y
    angle_rad = np.arctan2(dy, dx)
    angle_deg = np.degrees(angle_rad)
    angle_deg = (angle_deg + 360) % 360
    if angle_deg > 180:
        angle_deg = 360 - angle_deg
    return angle_deg
def get_activation_gesture(landmarks, handedness, threshold=200):
    thumb = landmarks[4]
    pinky = landmarks[20]
    thumb_x = thumb.x * 640
    pinky_x = pinky.x * 640
    if handedness == "Right":
        if (pinky_x - thumb_x) > threshold:
            return "down"
        elif (thumb_x - pinky_x) > threshold:
            return "up_activation"
        else:
            return "neutral"
    elif handedness == "Left":
        if (thumb_x - pinky_x) > threshold:
            return "down"
        elif (pinky_x - thumb_x) > threshold:
            return "up_activation"
        else:
            return "neutral"
    else:
        return "neutral"
def draw_basket(frame, x, y, color, angle=0):
    overlay = frame.copy()
    rect_top_left = (x - BASKET_WIDTH // 2, y - BASKET_HEIGHT // 2)
    rect_bottom_right = (x + BASKET_WIDTH // 2, y + BASKET_HEIGHT // 2)
    cv2.rectangle(overlay, rect_top_left, rect_bottom_right, color, -1)
    alpha = 0.6
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    return rect_top_left[0], rect_top_left[1]

def guardar_graficas(patient_name, session_id, angle_series, release_times, collected_count, failed_count, base_path):
    base_graficas = os.path.join(base_path, "graficas", patient_name)
    carpetas = {
        "angulo_vs_tiempo": os.path.join(base_graficas, "angulo_vs_tiempo"),
        "velocidad_volteo": os.path.join(base_graficas, "velocidad_volteo"),
        "aciertos_fallos": os.path.join(base_graficas, "aciertos_fallos"),
        "media_volteo": os.path.join(base_graficas, "media_volteo")
    }
    for ruta in carpetas.values():
        os.makedirs(ruta, exist_ok=True)

    tiempos, angulos = zip(*angle_series) if angle_series else ([], [])

    # 1. Ángulo en función del tiempo
    plt.figure()
    plt.plot(tiempos, angulos, label="Ángulo mano")
    plt.xlabel("Tiempo (s)")
    plt.ylabel("Ángulo (°)")
    plt.title("Ángulo de la mano vs Tiempo")
    plt.grid()
    plt.savefig(os.path.join(carpetas["angulo_vs_tiempo"], f"sesion_{session_id}.png"))
    plt.close()

    # 2. Velocidad de volteo para cada bola recogida
    if release_times:
        velocidades = [1 / t if t > 0 else 0 for t in release_times]
        plt.figure()
        plt.plot(range(len(velocidades)), velocidades, marker='o')
        plt.xlabel("Bola recogida")
        plt.ylabel("Velocidad de volteo (1/s)")
        plt.title("Velocidad de volteo por bola")
        plt.grid()
        plt.savefig(os.path.join(carpetas["velocidad_volteo"], f"sesion_{session_id}.png"))
        plt.close()

    # 3. Fallos y aciertos
    plt.figure()
    plt.bar(["Aciertos", "Fallos"], [collected_count, failed_count], color=['green', 'red'])
    plt.title("Aciertos vs Fallos")
    plt.savefig(os.path.join(carpetas["aciertos_fallos"], f"sesion_{session_id}.png"))
    plt.close()

    # 4. Media de volteo acumulativa
    historial_path = os.path.join(carpetas["media_volteo"], "media_volteo_acumulada.json")
    media_actual = np.mean([1/t for t in release_times if t > 0]) if release_times else 0
    historial = []
    if os.path.exists(historial_path):
        with open(historial_path, "r") as f:
            historial = json.load(f)
    historial.append(media_actual)
    with open(historial_path, "w") as f:
        json.dump(historial, f)

    plt.figure()
    plt.plot(range(1, len(historial)+1), historial, marker='o')
    plt.title("Media de volteo por sesión")
    plt.xlabel("Sesión")
    plt.ylabel("Media volteo (1/s)")
    plt.grid()
    plt.savefig(os.path.join(carpetas["media_volteo"], "media_volteo_acumulada.png"))
    plt.close()

# -------------------- CONSTANTES DEL JUEGO --------------------
WIDTH = 640
HEIGHT = 480
BASKET_WIDTH = 100
BASKET_HEIGHT = 30
BASKET_Y = HEIGHT // 2
CANDY_RADIUS = 10
THRESHOLD = 200

# -------------------- FUNCIÓN PRINCIPAL DEL JUEGO --------------------
# -------------------- FUNCIÓN PRINCIPAL DEL JUEGO --------------------
def start_game(params):
    import cv2
    import time
    import random
    import numpy as np

    patient_name = params['patient_name']
    falling_speed = params['falling_speed']
    total_balls = params['total_balls']

    conn, cursor, base_path = init_db()

    cursor.execute("SELECT id FROM patients WHERE name = ?", (patient_name,))
    result = cursor.fetchone()
    if result:
        patient_id = result[0]
    else:
        cursor.execute("INSERT INTO patients (name) VALUES (?)", (patient_name,))
        conn.commit()
        patient_id = cursor.lastrowid

    # Calcular media velocidad
    cursor.execute("""
        SELECT release_times FROM sessions
        WHERE patient_id = ? AND release_times IS NOT NULL AND release_times != '[]'
    """, (patient_id,))
    datos = cursor.fetchall()
    todas_las_velocidades = []
    for (rt,) in datos:
        tiempos = json.loads(rt)
        velocidades = [1 / t for t in tiempos if t > 0]
        todas_las_velocidades.extend(velocidades)
    media_velocidad_anterior = np.mean(todas_las_velocidades) if todas_las_velocidades else 0

    # Config Mediapipe
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    mp_draw = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

    release_times = []
    angle_series = []
    collected_count = 0
    failed_count = 0
    ball_counter = 0
    candy_start_time = time.time()
    punto_ganado = False
    punto_momento = 0

    # Calibración
    THRESHOLD = 200
    start_time_calib = time.time()
    calibration_time = 5
    calibrated = False

    while not calibrated:
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.flip(frame, 1)
        elapsed = time.time() - start_time_calib
        remaining = int(calibration_time - elapsed) + 1
        if remaining < 0:
            remaining = 0

        cv2.putText(frame, f"{remaining}", (WIDTH//2 - 30, HEIGHT//2 + 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
        cv2.putText(frame, "ABRE LA MANO TODO LO QUE PUEDAS", (30, HEIGHT//2 - 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if remaining == 0 and result.multi_hand_landmarks:
            hand_landmarks = result.multi_hand_landmarks[0]
            thumb = hand_landmarks.landmark[4]
            pinky = hand_landmarks.landmark[20]
            thumb_x, pinky_x = thumb.x * WIDTH, pinky.x * WIDTH
            THRESHOLD = int(abs(pinky_x - thumb_x))
            calibrated = True

            cursor.execute("SELECT rank, prestige, points FROM patients WHERE id = ?", (patient_id,))
            rank, prestige, points = cursor.fetchone()
            cv2.putText(frame, f"Rango: {rank}  Prestigio: {prestige}", (10, HEIGHT - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
            mostrar_imagen_medalla(rank, ventana=frame)
            cv2.imshow("Calibracion", frame)
            cv2.waitKey(2000)

        cv2.imshow("Calibracion", frame)
        if cv2.waitKey(1) == 27:
            break

    cv2.destroyWindow("Calibracion")

    # Juego
    candy_x = random.randint(CANDY_RADIUS, WIDTH - CANDY_RADIUS)
    candy_y = 0
    candy_state = "falling"
    score = 0
    last_basket_x = WIDTH // 2
    last_gesture = "neutral"
    basket_receptive = False

    cv2.namedWindow("Juego Ictus", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Juego Ictus", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks and result.multi_handedness:
            hand_landmarks = result.multi_hand_landmarks[0]
            handedness_label = result.multi_handedness[0].classification[0].label
            lm = hand_landmarks.landmark[8]
            basket_x = int(lm.x * WIDTH)
            gesture = get_activation_gesture(hand_landmarks.landmark, handedness_label, threshold=THRESHOLD)
            last_basket_x, last_gesture = basket_x, gesture
            current_time = round(time.time() - candy_start_time, 2)
            angle = calculate_hand_angle(hand_landmarks.landmark)
            angle_series.append((current_time, angle))
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        else:
            basket_x, gesture = last_basket_x, last_gesture

        basket_color = (0, 255, 0) if gesture == "up_activation" else (0, 0, 255) if gesture == "down" else (200, 200, 200)
        basket_angle = 0 if gesture != "down" else 180

        bx, by = draw_basket(frame, basket_x, BASKET_Y, basket_color, basket_angle)

        if candy_state == "falling":
            candy_y += falling_speed
            cv2.circle(frame, (candy_x, candy_y), CANDY_RADIUS, (0, 255, 255), -1)
            if gesture == "up_activation":
                basket_receptive = True
            if basket_receptive and gesture == "up_activation" and bx < candy_x < bx + BASKET_WIDTH and BASKET_Y - BASKET_HEIGHT//2 <= candy_y <= BASKET_Y + BASKET_HEIGHT//2:
                candy_state = "collected"
                collected_time = time.time()

            if candy_y > HEIGHT:
                failed_count += 1
                ball_counter += 1
                candy_x, candy_y = random.randint(CANDY_RADIUS, WIDTH - CANDY_RADIUS), 0

        elif candy_state == "collected":
            candy_x = basket_x
            candy_y = BASKET_Y - BASKET_HEIGHT//2 - CANDY_RADIUS
            cv2.circle(frame, (candy_x, candy_y), CANDY_RADIUS, (0, 255, 255), -1)
            if gesture == "down":
                tiempo_volteo = time.time() - collected_time
                release_times.append(tiempo_volteo)
                collected_count += 1
                candy_state = "released"
                last_speed = 1 / tiempo_volteo if tiempo_volteo > 0 else 0
                if last_speed > media_velocidad_anterior:
                    punto_ganado = True
                    punto_momento = time.time()

        elif candy_state == "released":
            candy_y += falling_speed * 5
            cv2.circle(frame, (candy_x, candy_y), CANDY_RADIUS, (0, 255, 255), -1)
            if candy_y > HEIGHT:
                score += 1
                ball_counter += 1
                candy_state = "falling"
                candy_x, candy_y = random.randint(CANDY_RADIUS, WIDTH - CANDY_RADIUS), 0
                basket_receptive = False

        cv2.putText(frame, f"Score: {score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(frame, f"Cogidas: {collected_count}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.putText(frame, f"Fallos: {failed_count}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        cv2.putText(frame, f"Bolas: {ball_counter}/{total_balls}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        if punto_ganado and (time.time() - punto_momento) < 1.0:
            cv2.putText(frame, "+1", (WIDTH - 100, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
        else:
            punto_ganado = False

        cv2.imshow("Juego Ictus", frame)
        if cv2.waitKey(1) == 27 or ball_counter >= total_balls:
            break

    cap.release()
    cv2.destroyAllWindows()

    cursor.execute("""
        INSERT INTO sessions (patient_id, timestamp, release_times, collected_count, failed_count, falling_speed, total_balls, angle_series)
        VALUES (?, datetime('now'), ?, ?, ?, ?, ?, ?)
    """, (patient_id, json.dumps(release_times), collected_count, failed_count, falling_speed, total_balls, json.dumps(angle_series)))
    session_id = cursor.lastrowid
    conn.commit()

    guardar_graficas(patient_name, session_id, angle_series, release_times, collected_count, failed_count, base_path)

    nuevo_rango, nuevo_prestigio, total = actualizar_rango_y_puntos(cursor, conn, patient_id, release_times)
    mostrar_mensaje_resultado(total)
    mostrar_imagen_medalla(nuevo_rango)
    conn.close()

    messagebox.showinfo("Fin del Juego", f"Resultados:\nBolas cogidas: {collected_count}\nBolas falladas: {failed_count}\nScore: {score}")

# -------------------- INTERFAZ DE CONFIGURACIÓN --------------------
def launch_interface():
    root = tk.Tk()
    root.title("Configuración del Juego Ictus")
    root.geometry("400x350")

    params = {}

    tk.Label(root, text="Nombre del paciente:", font=("Helvetica", 12)).pack(pady=10)
    entry_name = tk.Entry(root, font=("Helvetica", 12))
    entry_name.pack(pady=5)

    tk.Label(root, text="Velocidad de caída:", font=("Helvetica", 12)).pack(pady=10)
    speed_var = tk.StringVar(value="5")
    for text, val in [("Lento (3)", "3"), ("Medio (5)", "5"), ("Rápido (8)", "8")]:
        tk.Radiobutton(root, text=text, variable=speed_var, value=val, font=("Helvetica", 11)).pack(anchor="w", padx=20)

    tk.Label(root, text="Total de bolas:", font=("Helvetica", 12)).pack(pady=10)
    entry_total = tk.Entry(root, font=("Helvetica", 12))
    entry_total.pack(pady=5)
    entry_total.insert(0, "10")

    def iniciar_juego():
        name = entry_name.get().strip()
        if not name:
            messagebox.showerror("Error", "Debes ingresar un nombre.")
            return
        params['patient_name'] = name
        try:
            params['falling_speed'] = int(speed_var.get())
            params['total_balls'] = int(entry_total.get())
        except:
            messagebox.showerror("Error", "Valores inválidos.")
            return
        root.destroy()
        start_game(params)

    tk.Button(root, text="Iniciar Juego", command=iniciar_juego,
              font=("Helvetica", 14, "bold"), bg="#4CAF50", fg="white", padx=10, pady=5).pack(pady=20)

    root.mainloop()

# -------------------- EJECUCIÓN --------------------
if __name__ == "__main__":
    launch_interface()
